In [1]:
import numpy as np
import math as mth
import random
from collections import deque

In [2]:
def GeneticAlgorithm(fun, funInEq, funEq, IntIndx, lb, ub):
    n = len(lb); popsize = 50; penalty = 100; ubmlb = ub - lb; r = [0];
    Xreal = np.zeros((n, popsize*2)); Xbin = []; NotIntIndx = []; 
    
    for i in range(n):
        isin = i in IntIndx;
        if(not isin):
            NotIntIndx.append(i);
    
    for j in range(popsize*2):
        Xbin.append('rand');
    
    for i in range(n):
        l = mth.ceil(mth.log2(ubmlb[i]));
        if (l == 0):
            l = 1;
        if(i in IntIndx):
            r.append(r[i] + l);
        else:
            r.append(r[i] + l + 10);
    
    def Objfun(j):
        Of = fun(Xreal[:, j]); 
        ineq = np.max(0, funInEq(Xreal[:, j])).sum();
        eq = np.abs(funEq(Xreal[:, j])).sum();
        return Of + penalty*(ineq + eq);
    
    def rand(j):
        for i in IntIndx:
            Xreal[i, j] = random.randint(lb[i], ub[i])
        for i in NotIntIndx:
            Xreal[i, j] = lb[i] + random.random()*ubmlb[i];
        
    def real2bin(j):
        Xbin[j] = '';
        for i in NotIntIndx:
            nb = r[i + 1] - r[i]; den = 2**nb;
            xres = mth.floor((Xreal[i, j] - lb[i])*den/ubmlb[i]);
            binary = bin(xres)[2:].zfill(nb);
            Xbin[j] += binary;  
        for i in IntIndx:
            nb = r[i + 1] - r[i];
            xres = Xreal[i, j]  - lb[i];
            binary = bin(xres)[2:].zfill(nb);
            Xbin[j] += binary;  
            
    def bin2real(j):
        for i in NotIntIndx:
            str = Xbin[j][r[i]:r[i + 1]];
            nb = r[i + 1] - r[i]; den = 2**nb - 1;
            xres = int(str, 2);
            Xreal[i, j] = lb[i] + xres*ubmlb[i]/den; 
        for i in IntIndx:
            str = Xbin[j][r[i]:r[i + 1]];
            nb = r[i + 1] - r[i]; den = 2**nb;
            xres = int(str, 2);
            Xreal[i, j] = int(mth.ceil(lb[i] + xres*ubmlb[i]/den)); 
            
    def Reproduce(i, j, k, Order):
        crp = random.randint(1, r[-1]-1);
        Xbin[Order[popsize + k*2]] = Xbin[Order[i]][:crp] + Xbin[Order[j]][crp:];
        Xbin[Order[popsize + k*2 + 1]] = Xbin[Order[j]][:crp] + Xbin[Order[i]][crp:];
    
    def Mutate(i):
        k = random.randint(1, r[-1]-1); kp1 = k+1;
        Xbin[i] = Xbin[i][:k] + str((int(Xbin[i][k]) + 1) % 2) + Xbin[i][kp1:];
        
    ans = GeneticAlgorithmWorkHorse(Objfun, real2bin, bin2real, Reproduce, Mutate, rand, n, Xreal, Xbin, popsize, lb, ub, ubmlb);
    return ans;

In [3]:
def GeneticAlgorithmWorkHorse(Objfun, real2bin, bin2real, Reproduce, Mutate, rand, nvar, Xreal, Xbin, popsize, lb, ub, ubmlb):
    imax = int(round(popsize/2)); popsize2 = popsize*2; w = -1; AvegVals = deque(); k1 = 0.5; k2 = 0.5; k3 = 0.5; k4 = 0.5;
    pi = mth.pi; Fval = np.zeros(popsize*2); Fits = np.zeros(popsize2); Bfval = mth.inf; sol = np.zeros(nvar); Wfval = -mth.inf;
    
    # Fitness function
    def FitFun(f):
        fitfun = 0.5*(mth.atan(-f)/pi + 1);
        return fitfun;
    
    # Function pointer for sorting
    def takeSecond(elem):
        return elem[1];
    for i in range(popsize):
        # Generate Random Solution and Convert2Binary
        rand(i); real2bin(i);
        # Evaluate Solution
        Fval[i] =  Objfun(i);
        # Compare Solution to Best Solution Ever Found
        if (Fval[i] < Bfval):
            Bfval = Fval[i];
            sol = Xreal[:,i];
        # Compare Solution to Worst Solution Ever Found
        if (Fval[i] > Wfval):
            Wfval = Fval[i]; w = i; 

    # Populate the popsize to 2 popsize
    for i in range(popsize, popsize2):
        Xreal[:,i] = Xreal[:,w]; 
        Xbin[i] = Xbin[w]; 
        Fval[i] = Fval[w];

    # Ranking Initial Population
    Pair = []; Order = [];
    for i in range(popsize2):
        Pair.append((i, Fval[i]));

    Pair.sort(key=takeSecond);
    for i in range(len(Fval)):
        Order.append(Pair[i][0]);
        Fits[i] = FitFun(Pair[i][1]);

    fb = Fits[:popsize].sum() / popsize; fmax = Fits[0]; AvegVals.append(fb);
    iter = 0; maxiter = 800; IsMinimized = 0; Tol = 1e-6;  
    while (iter < maxiter):
        # Selection/Reproduction (Elitism)
        for k in range(imax):
            i = random.randint(0, popsize); j = random.randint(0, popsize);
            fp = max(Fits[i], Fits[j]); pc = k3;
            if(fp >= fb):
                pc = k1 * (fmax - fp) / (fmax - fb);
            if(random.random() < pc):
                Reproduce(i, j, k, Order);

        # Convert to Real, Evaluate and Enqueue
        for i in range(popsize2):
            bin2real(i); Fval[i] = Objfun(i);
            if (Fval[i] < Bfval):
                Bfval = Fval[i]; sol = Xreal[:,i];


        # Sort to keep the next generation at the beginning
        Pair = []; Order = []; 
        for i in range(len(Fval)):
            Pair.append((i, Fval[i]));

        Pair.sort(key=takeSecond);
        for i in range(len(Fval)):
            Order.append(Pair[i][0]);
            Fits[i] = FitFun(Pair[i][1]);

        fb = Fits[:popsize].sum() / popsize; fmax = Fits[0]; AvegVals.append(fb);

        # Genetic Alteration (Mutation)
        for i in range(popsize):
            fi = Fits[i]; pm = k4;
            if(fi >= fb):
                pm = k2 * (fmax - fi) / (fmax - fb);
            if (random.random() < pm):
                Mutate(Order[i]);  

        # Check convergence
        if (iter > nvar):
            AvegmN = AvegVals.popleft();
            IsMinimized = mth.fabs(fb - AvegmN) < Tol;
            if (IsMinimized):
                break;

        iter += 1;
    print("iter = " + str(iter))
    return sol;

In [4]:
mp = -6.5511;
Lb = -3*np.ones(2); Ub = Lb+6;
def peak(v):
    x = v[0]; y = v[1]; x2 = x**2; y2 = y**2; xp1 = x + 1; yp1 = y + 1; x3 = x**3;
    y5 = y**5; xm1 = 1 - x; xm1_2 = xm1**2; yp1_2 = yp1**2; xp1_2 = xp1**2;
    f = 3 * xm1_2 * mth.exp(-x2 - yp1_2) - 10 * (x / 5 - x3 - y5) * mth.exp(-x2 - y2) - 1.0 / 3 * mth.exp(-xp1_2 - y2);
    return f;
def funInEq(v):
    f = 0;
    return f;
def funEq(v):
    f = 0;
    return f;
CorrecCount = 0;
for n in range(50):
    sol = GeneticAlgorithm(peak, funInEq, funEq, [], Lb, Ub);
    
    print(sol); fv = peak (sol); print(fv);
    if(mth.fabs((fv-mp)/mp) < 1e-3):
        CorrecCount += 1;
print("CorrecCount = " + str(CorrecCount));

iter = 114
[ 0.22890978 -1.62727384]
-6.5510817401972945
iter = 135
[ 0.42082774 -1.57453302]
-6.199085018760353
iter = 95
[ 0.22671225 -1.62580881]
-6.55111028076195
iter = 52
[ 0.25161763 -1.62287877]
-6.546032677008375
iter = 87
[ 0.22964229 -1.6250763 ]
-6.551114790991558
iter = 107
[ 0.24062996 -1.62727384]
-6.549545288421378
iter = 145
[ 0.22964229 -1.62800635]
-6.551014684511001
iter = 212
[ 0.22744476 -1.62361128]
-6.551067088455232
iter = 118
[ 0.22671225 -1.62873886]
-6.550980588858548
iter = 77
[ 0.23843243 -1.62580881]
-6.550131699835198
iter = 150
[ 0.22890978 -1.62873886]
-6.550973843352714
iter = 103
[ 0.20253937 -1.60603101]
-6.537209207506326
iter = 86
[ 0.2237822  -1.63386644]
-6.550079971907869
iter = 52
[ 0.22451471 -1.62361128]
-6.5509160390786185
iter = 91
[ 0.21865462 -1.63459895]
-6.549389402349194
iter = 123
[ 0.21865462 -1.62873886]
-6.5502120869552565
iter = 90
[ 0.22817727 -1.61921621]
-6.5505531697069195
iter = 117
[ 0.22964229 -1.62287877]
-6.5510272309399

In [7]:
CorrecCount = 0;
Lb = np.zeros(2); Ub = Lb+2;
def rosenbrock (v):
    s = 0;
    for n in range(1, len(v)):
        x = v[n-1]; y = v[n];
        s += 100*(y-x**2)**2 + (1-x)**2;
    return s;
def funInEq(v):
    f = 0;
    return f;
def funEq(v):
    f = 0;
    return f;
for n in range(50):
    sol = GeneticAlgorithm(rosenbrock, funInEq, funEq, [], Lb, Ub);
    print(sol); fv = rosenbrock (sol); print(fv);
    if(mth.fabs(fv) < 1e-3):
        CorrecCount += 1;
print("CorrecCount = " + str(CorrecCount));

iter = 800
[1.0073278  1.01319003]
0.00028451047301171186
iter = 608
[0.91548608 0.84123107]
0.008113743407368663
iter = 800
[1.03370787 1.06399609]
0.003211804995938928
iter = 689
[1.01025892 1.02296043]
0.0006515675219523588
iter = 800
[0.98680997 0.97313141]
0.00021786720140061283
iter = 203
[1.00928188 1.02491451]
0.004010679529591862
iter = 800
[0.9926722  0.98583293]
7.26037253269315e-05
iter = 800
[1.01319003 1.02784563]
0.00034079549957337677
iter = 800
[1.12554958 1.29164631]
0.07718956430656981
iter = 410
[0.96140694 0.92818759]
0.0029981978192429566
iter = 191
[1.02198339 1.05617978]
0.014241914720730092
iter = 800
[0.96726917 0.93404983]
0.0013146128869324053
iter = 127
[0.96140694 0.93698095]
0.017561703572149836
iter = 800
[0.9457743  0.90083048]
0.006961824670965351
iter = 576
[0.95847582 0.92232535]
0.003056113473564488
iter = 621
[0.89594529 0.81582804]
0.02801481882555319
iter = 539
[0.99462628 0.99853444]
0.008590676028955246
iter = 523
[1.00439668 1.00830484]
4.5121